In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def web_search(query: str) -> dict[str, Any]:
    """Search the web for information."""
    return tavily_client.search(query)

@tool
def sql_query(query: str) -> dict[str, Any]:
    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [3]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [4]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from collections.abc import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
)

agent = create_agent(
    model=model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

[{'type': 'text', 'text': 'There are 275 artists in the database.', 'extras': {'signature': 'CtQBAXLI2nwWYiohsB8oenAEMcfTf9TTWIfgmKBUFIWJFwNZmyr5OWmREwfg5igPnjjrIndYuH8EiCrIv0yA0z80NwPrUltQ5uncW7k5vPZuWQS5wrCkG1NBMhUIrUqaZqSsi+4ZBbwaLySgpNgEuZDJ9C8gqel8rdh93cpGDXxZ4dg3UvPK9QSYUkWvAgZBWaa4WxYU46h7rHpsI5i5yMhC7l9meP+YuB3muabDsQya2jskgU8UBnFTDbx7jI0+g8pTr4VgxqAaAhKQKhWAvrmuxf9ouJE='}}]


In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

[{'type': 'text', 'text': 'I do not have access to a database to provide you with that information.', 'extras': {'signature': 'CqcKAXLI2nyo2p5O1UwsQuy0e+M6BlOYdfF2nUmgSdwnugaMp2wmjwZI53UN2MVoTdftdoH4sa6tUZBEqP13y+Vj7mVr9A/YzFBiz5xMrg7TpHwP/B3jlGoY1GwbzC7kFK0O+943NDKn6Rd9qKvw+FtLutjjlViZHzcfKHaN830k8bBIWHqTsrt9Y/neG3aS7HPR2FnnVEkZMpZuE0jB8mUDmHIoK+3AtEUjSZV1HZukm1ovVG3DCU7I6Y738NXMKtcEAF+PaNTCzneREJvl58xDGmlHAVOrcUCX75tPVReA/i1S7e3eknSHHrNXhAeMmluyB6r5nvNC3YSLfh9Zu3kYB3G5I0CfA9wWzT8C7oWzuaI+J5joeWgxeJf/s0uybzeSgv9YA5svQdQIwhBtSOeelHaKCtEl5/9Ku9J7gj1XbPAzlMU3V2DMNvR528cNuwoi8T05ksK3gHLhO6vsIAbqfCAvIJdFJsKKoMXNGyFakGd3Yy+HLr17NlXZ1+6sj9bwswWRRm2TAJEhzYuMFQCeXuFGKqd3dspYUBwC2Xb1jVG3dIPbtRf4QYBbtmjNEU/jo27DK7lJPceK7qtB1pPI/aqQPMatPXbWYdB7DSdBWQbLUB6oaBPkDoynrGQLi9SwTfD/LhNxblCz0CMeMlb3Cd013c9Z2xdjkM9f18CUWvpN+snaIEormhiwg1+rDEDrTrX14xJdPp7OylTkdSS1slR4aCLuFzBaKVk92TlBnfXZa3TGqvSeuXPIzMmNurrUxYTkEsnXI9JP/KaiOi1EkeFNBfrtBFzI6SqSKu0JcN+Vld5JYzwSfJFdkOXzHWY1D0hPEtlmJ/cAMtIWSw7Z6YPnGUYFj/hEj3KyI